Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Aseem Sachdeva"
COLLABORATORS = ""

# Presenting Uncertainty
## School of Information, University of Michigan

## Week 1: Assignment Overview
Version 1.1
### The objectives for this week are for you to:
- Visualize a population distribution, sample, and sampling distribution
- Practice visualizing density plots and histograms
- Practice bootstrapping a sampling distribution
- Review and reflect on the concept and motivation of bootstrap

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import altair as alt
from collections import Counter

# Part 1. Visualize the bootstrap of a single mean (15 points)

Suppose that you want to summarize how many times a day students at a university pick up their smartphones. But at a university of a reasonable size (say 5,000 students), it would be hard to take a complete **census** of everyone at the school. Instead, you make an online survey which points responders to a pickup-counting app they can install on their phone to help track pickups. In the next few days, you receive 30 students' responses with their number of pickups in a given day, a **sample** of all students in school. You will calculate the mean of these 30 pickups to get an **estimate** for the total number of pickups in the lab.

What we would like to know is the **population mean**: the average number of times per day students at this university pick up their phones. This is a **population parameter**; hence our uncertainty in it is **parameter uncertainty**.

Since it is infeasible to collect data from all 5,000 students, we'll have to estimate that number based on a smaller **sample**. We also want to **quantify our uncertainty** in this estimate using some kind of distribution, as discussed in lecture. While there are many ways to do this (confidence distributions, Bayesian posterior distributions, etc), for the purposes of this assignment we will construct a **bootstrap sampling distribution**. Below is a python example to work through. (5 points)

### Question 1.1.1 Generating the population (2 points)

For the purposes of this example, we will first generate the population we want to sample from. Remember that in reality, we would not be generating the population, nor would we be able to observe the population directly! But for this first example, we want to explore the relationship between the population, the sample, and between the population mean and an estimate of the mean. So we will start with a known population, a luxury we do not have in real-world analyses.

Fill in the code below to construct a function to generate the population.

In [3]:
def get_population(lam, size=5000, seed=42):
    
    """
    Construct a function to generate the population for the phone pickup scenario.
    The population should be randomly drawn from a Poisson distribution with a mean
    of `lam`. Use the np.random.poisson() function to generate the data.
    Hist: the `lam` parameter of np.random.poisson() is the mean of the Poisson distribution.
    
    lam: the mean (lambda parameter of a Poisson distribution) used to generate the population
    size: the total number of people in the population (default: 5000 people).
    seed: seed used to initialize the random number generator for reproducibility (default: 42).
    
    This function should return the following variables as a tuple: (pickups, mean, std).
    pickups: a numpyarray of length `size` where each item is a randomly generated
        total pickup from a Poisson distribution with mean `true_mean`
    mean: the population mean number of pickups
    std: the population standard deviation of pickups
    """
    
    np.random.seed(seed)
    
    pickups = np.random.poisson(lam= lam, size=size)
    mean = np.mean(pickups)
    std = np.std(pickups)
    

    
    return pickups, mean, std

In [4]:
# pop_pickups = get_population(45, 5000, 42)
# pop_pickups

In [5]:
# test your code, we want to check that population is correctly generated before continuing
pop_pickups, pop_mean, pop_std = get_population(45, 5000, 42)
assert np.abs(pop_mean - 45) < 0.5, "Get population problem, testing pop_mean, population mean returned does not match expected population mean"
assert np.abs(pop_std - np.sqrt(45)) < 0.5, "Get population problem, testing pop_std, population standard deviation returned does not match expected standard deviation"

Using the above function, you can construct a population and inspect the population mean and standard deviation as follows:

In [6]:
pop_pickups, pop_mean, pop_std = get_population(45, 5000)
print("Population mean:", pop_mean)
print("Population SD:", pop_std)

Population mean: 45.0598
Population SD: 6.786886175559452


### Question 1.1.2 Sampling from the population (2 points)

Next, we'll generate a **sample** from the population. Fill in the `get_sample_statistics()` function:

In [7]:
def get_sample_statistics(pickups, size, seed=42):
    
    """
    Construct a function to draw a random sample from a population for the phone pickup scenario.
    
    pickups: the numpyarray containing the full population; e.g. the first element in the tuple
        generated by `get_population()`
    size: the number of people included in the sample pulled from the total population.
        For example, it may be 30 students.
    seed: seed used to initialize the random number generator for reproducibility (default: 42).
    
    This function should return the following variables as a tuple: (sample, mean, std)
    sample: a numpyarray where each item is a randomly selected from the total population.
    mean: the mean of the sample
    std: the standard deviation of the sample
    """
    
    np.random.seed(seed)
    
    sample = np.random.choice(pickups, size)
    mean = np.mean(sample)
    std = np.std(sample)

    
    return sample, mean, std

In [8]:
# test your code, we want to check that the statistics for sample data looks correct before we continue doing more exploration
pop_pickups, pop_mean, pop_std = get_population(45, 5000, 42)
sample_pickups, sample_mean, sample_std = get_sample_statistics(pop_pickups, 30, 42)
assert np.abs(sample_mean - 44) < 0.5, "Get sample statistics problem, testing sample_mean, sample mean returned does not match expected sample mean"
assert np.abs(sample_std - 6) < 0.5, "Get sample statistics problem, testing sample_std, sample standard deviation returned does not match expected standard deviation"

Using the above function, you can construct a sample from the population as follows:

In [9]:
pop_pickups, pop_mean, pop_std = get_population(45, 5000)
sample_pickups, sample_mean, sample_std = get_sample_statistics(pop_pickups, 30)
print("Sample mean:", sample_mean)
print("Sample SD:", sample_std)

Sample mean: 44.333333333333336
Sample SD: 5.605552802554109


## 1.3 Visualize and compare population distribution, sample, and bootstrap sampling distribution
The population distribution shows the values of the variable for all the individuals in the population. The distribution of sample data contains the values of the variable for all the individuals in the sample, which is a subset of the population.

For a reminder of the ideas behind sampling and confidence intervals, go to [Seeing Theory Chapter 2](https://seeing-theory.brown.edu/frequentist-inference/index.html#section2).

We'll start by visualizing the population distribution:

In [10]:
df = pd.DataFrame(data = {"Pickups" : pop_pickups})

pop_hist_chart = alt.Chart(df).mark_bar().encode(
    alt.X("Pickups", bin=alt.Bin(maxbins=40), 
        # set the x axis limits to be from 0 to 80
        # so that it is easier to compare across visualizations.
        scale=alt.Scale(domain=[0,80])  
    ),
    y='count()'
)

pop_hist_chart

alt.Chart(...)

Aternatively, we could map population density onto the y position by using the `transform_density()` method in Altair. Combined with an `area` mark, this allows us to create a density plot. We'll also add a dotted vertical rule indicating the **population mean** in black:

In [11]:
df = pd.DataFrame(data = {"pop_pickups" : pop_pickups})

pop_density_chart = alt.Chart(df).transform_density(
    density='pop_pickups',
    as_=['pop_pickups', 'density'],
    
).mark_area(color="lightgray").encode(
    x=alt.X('pop_pickups', axis=alt.Axis(title="Population Distribution of Pickups")),
    y=alt.Y('density:Q', axis=alt.Axis(title="Density"))
)

pop_mean_chart = alt.Chart(df).mark_rule(color="black", strokeDash=[1,1]).encode(
    x="mean(pop_pickups)"
)

pop_chart = pop_density_chart + pop_mean_chart

pop_chart

alt.LayerChart(...)

We can also look at a dot plot of the sample drawn from the population. We'll add a vertical rule indicating the **sample mean** in red:

In [12]:
df = pd.DataFrame(data = {"sample_pickups" : sample_pickups})

sample_dot_chart = alt.Chart(df).mark_circle(color="gray").encode(
    alt.X("sample_pickups", axis=alt.Axis(title="Sample of 30 Pickups"))
)

sample_mean_chart = alt.Chart(df).mark_rule(color="red").encode(
    x="mean(sample_pickups)"
)

sample_chart = sample_dot_chart + sample_mean_chart + pop_mean_chart

sample_chart

alt.LayerChart(...)

### Question 1.3.1 Sampling distribution of the mean (4 points)

As mentioned in this week's lecture, one way to quantify the uncertainty in an estimate is through a **bootstrap sampling distribution**. To derive a bootstrap sample distribution for a statistic (say a mean) from a sample of size *k* (here, *k* = 30), follow these steps *B* times, to get *B* statistics, forming samples from your bootstrap sampling distribution:

1. Draw random sample of size *k* with replacement from your existing sample.
2. Compute the statistic (e.g. the mean) for the sample.

We then choose a large value of *B* (say 5000) to approximate the sampling distribution of the desired statistic (e.g., the mean).

Follow the instructions below, within the code cell, to construct the bootstrap sampling distribution for the mean of your sample (i.e. the `sample_pickups` variable):

In [13]:
#Number of bootstrap samples --- typically something large, like 5000
B = 5000

# set the random seed for reproducibility
np.random.seed(42)

bootstrap_means = []
# construct the bootstrap sampling distribution of the mean of you sample (`sample_pickups`)
# and store it in `bootstrap_means`
# Hint: use np.random.choice()
# YOUR CODE HERE


Replications = np.array([np.random.choice(sample_pickups, size=30, replace = True) for i in range(B)])
bootstrap_means = np.mean(Replications, axis=1)






print("Population mean:        ", pop_mean)
print("Sample mean:            ", sample_mean)
# the bootstrap sampling mean should be similar to the mean of your sample
print("Bootstrap sampling mean:", np.mean(bootstrap_means))

Population mean:         45.0598
Sample mean:             44.333333333333336
Bootstrap sampling mean: 44.35646666666666


In [14]:
# the bootstrap sampling mean should be close to the sample mean
sample_mean_se = np.std(sample_pickups, ddof=1) / np.sqrt(len(sample_pickups))
assert len(bootstrap_means) == 5000, "Bootstrap sampling distribution: length of bootstrap_means should be 5000"
assert np.abs(np.std(bootstrap_means) - sample_mean_se) / sample_mean_se < 0.05, "Bootstrap sampling distribution: SD of boostrap_means does not match SE of sample_pickups"
assert np.abs(np.mean(bootstrap_means) - sample_mean)/sample_mean < 5*1e-3, "Bootstrap sampling distribution: mean of bootstrap_means does not match sample_mean"

### Question 1.3.2 Combined plot of population, sample, and sampling distribution (7 points)

Finally, plot your population distribution, sample, and the sampling distribution of the mean all together. Your output should look like this:

![Plot showing the population distribution, a sample of size 30, and a bootstrap sampling distribution of the mean stacked on top of each other](assets/assignment1_pop_sample_bootstrap.png)

In [15]:
# Hints: use the `.properties()` method to adjust width/height of plots, and
# use `.resolve_scale()` to make the x axes line up with each other. 
# `alt.vconcat()` (or the `&` operator) can be used to stack charts.

# YOUR CODE HERE






bootstrap_df = pd.DataFrame(data = {"bootstrap_means" : bootstrap_means})

pop_density_chart_bootstrap = alt.Chart(bootstrap_df).transform_density(
    density='bootstrap_means',
    as_=['bootstrap_means', 'density'],
    
).mark_area(color="lightgray").encode(
    x=alt.X('bootstrap_means', axis=alt.Axis(title="Bootstrap sampling Distribution of mean Pickups")),
    y=alt.Y('density:Q', axis=alt.Axis(title="Density"))
)



pop_chart_bootstrap = pop_density_chart_bootstrap + pop_mean_chart + sample_mean_chart

pop_chart_bootstrap = pop_chart_bootstrap.properties(height=200)

pop_chart = pop_chart.properties(height=150)


combined_vertical_chart = alt.vconcat(pop_chart, sample_chart, pop_chart_bootstrap)

combined_vertical_chart = combined_vertical_chart.resolve_scale(
            x='shared'
            )


combined_vertical_chart

alt.VConcatChart(...)

# Part 2. Boston Housing Prices (15 points)

Practice: In this section you will predict the price of houses in Boston using linear regression.

Linear regression models the relationship between a response variable (sometimes called a dependent variable) and one or more predictors (sometimes called independent variables) as a linear function, like this:

$\begin{align}
y_i &\sim \mathrm{Normal}(\mu_i, \sigma^2)\\
\mu_i &= \theta_{0} + \theta_{1}x_{1,i} + \cdots + \theta_{p}x_{p,i}
\end{align}
$

Where:

- $y_i$ is observation $i$
- $x_{p,i}$ is predictor $p$ for observation $i$
- $\theta_p$ is the *coefficent* for predictor $p$
- $\theta_0$ is the intercept

We would like to estimate those coefficients and quantify their uncertainty using bootstrapping.

### Reminder: Large-world versus small-world uncertainty

We are going to use ordinary least squares (OLS) linear regression, which has several assumptions behind it. These include things like *constant variance* (also called homoskedasticity), which means that the variance of observations ($\sigma^2$ in the above formula) is not a function of the predictors (the $x$s); and that the effects of predictors on the mean of the response are *linear* and *additive*. The validity of these assumptions is related to large-world uncertainty, and should be evaluated in real world applications. We will return to large-world uncertainty later in the course.

## 2.1 Load and explore the data

As stated above, we will be using the Boston Housing Price dataset. 

We're going to use two variables from the dataset as predictors:

- **RM**: average number of rooms per dwelling
- **LSTAT**: average of the proportion of adults without some high school education and the proportion of male workers classified as laborers

Our response variable will be the **price**.

Let's load the dataset and view some descriptive statistics.

In [16]:
#Load the Boston Housing Data Set from sklearn.datasets
boston = load_boston()

In [17]:
#create a dataframe containing predictors (housing_X) and the response variable (housing_y)
housing_X = pd.DataFrame(boston.data, columns = boston.feature_names)[['RM', 'LSTAT']]
housing_y = pd.Series(boston.target, name = "price")

#for exploration, also create a combined data frame with both predictors and response variables
housing = pd.concat([housing_y, housing_X], axis=1)

# show the first ten rows of the data we'll be using
housing.head(n = 10)

,price,RM,LSTAT
0,24.0,6.575,4.98
1,21.6,6.421,9.14
2,34.7,7.185,4.03
3,33.4,6.998,2.94
4,36.2,7.147,5.33
5,28.7,6.430,5.21
6,22.9,6.012,12.43
7,27.1,6.172,19.15
8,16.5,5.631,29.93
9,18.9,6.004,17.10


In [18]:
#Get some statistics from our variables: count, mean standard deviation etc.
housing.describe()

,price,RM,LSTAT
count,506.000000,506.000000,506.000000
mean,22.532806,6.284634,12.653063
std,9.197104,0.702617,7.141062
min,5.000000,3.561000,1.730000
25%,17.025000,5.885500,6.950000
50%,21.200000,6.208500,11.360000
75%,25.000000,6.623500,16.955000
max,50.000000,8.780000,37.970000


Many factors may have impact on housing price! It's usually a good idea to generate a data visualization to show the relationship between your variables. Below, we visualize the relationship between the two variables in the dataset (`RM` and `LSTAT`) and the housing price (`price`):

In [19]:
base = alt.Chart(housing).mark_circle(color="black").encode(
    alt.X("RM"), alt.Y("price")
)

# Create regression line
linear_fit = base.transform_regression(
    "RM", "price", method="linear"
).mark_line()

base + linear_fit

alt.LayerChart(...)

In [20]:
base = alt.Chart(housing).mark_circle(color="black").encode(
        alt.X("LSTAT"), alt.Y("price")
)

# Create regression line
linear_fit = base.transform_regression(
    "LSTAT", "price", method="linear"
).mark_line()

base + linear_fit

alt.LayerChart(...)

From the plots above, we see that both `RM` and `LSTAT` are correlated with `price`. Let's fit a linear model using these explanatory variables to predict average housing price.

## 2.2 Train a linear model

A linear model for `price` based on `RM` and `LSTAT` might look like this:

$\begin{align}
\mathrm{price}_i &\sim \mathrm{Normal}(\mu_i, \sigma^2)\\
\mu_i &= \theta_{0} + \theta_{\mathrm{RM}}\mathrm{RM}_{i} + \theta_{\mathrm{LSTAT}}\mathrm{LSTAT}_{i}
\end{align}
$

Where:

- $\mathrm{price}_i$ is the `price` on row $i$
- $\mathrm{RM}_{i}$ is the value of `RM` on row $i$
- $\mathrm{LSTAT}_{i}$ is the value of `LSTAT` on row $i$
- $\theta_\mathrm{RM}$ is the *coefficent* for `RM`
- $\theta_\mathrm{LSTAT}$ is the *coefficent* for `LSTAT`
- $\theta_0$ is the intercept
- $\sigma$ is the standard deviation of the price conditional on the predictors

### Question 2.2.1 Linear regression (5 points)

- Complete the `linear_reg` function below to return the model above given a data frame of predictors (`X`) and a data frame with the response variable (`y`). You should use sklearn's class: `linear_model.LinearRegression`.

In [21]:
from sklearn.linear_model import LinearRegression

def linear_reg(X, y):
    """
    X: A pandas DataFrame containing the predictors as columns
    y: A pandas Series containing the response variable
    
    step 1: Initialize the linear regression model
    step 2: Fit the model on X and y
    """
    # YOUR CODE HERE
 
    model = LinearRegression().fit(housing_X, housing_y)


  
    return model

In [22]:
reg = linear_reg(housing_X, housing_y)

In [23]:
# test your linear regression code
reg = linear_reg(housing_X, housing_y)
assert len(reg.coef_) == 2, "Implement linear regression problem, testing the number of coefficients, number of coefficients returned does not match expected, check your linear regression code"
assert np.abs(reg.coef_[0] - 5) < 1, "Implement linear regression problem, testing the RM coefficient, RM coefficient returned does not match expected, check your linear regression code"
assert np.abs(reg.coef_[1] - 0) < 1, "Implement linear regression problem, testing the LSTAT coefficient, LSTAT coefficient returned does not match expected, check your linear regression code"

The coefficients below give us an estimate of the impact of a one-point increase in the predictor on the housing price---*contingent upon our model assumptions matching well with the real world*.

But remember that there is uncertainty in these estimates!

In [24]:
#Print the coefecients/weights for each feature/column of our model
print("room number coefficient:     %.2f" % (reg.coef_[0]))
print("lower status coefficient:    %.2f" % (reg.coef_[1]))

room number coefficient:     5.09
lower status coefficient:    -0.64


## 2.3 Derive small-world uncertainty using bootstrapping

In our bootstrapping methods and analysis, we will focus on the coefficient for average number of rooms, $\theta_{RM}$. We would like to explore the partial relationship between price and number of rooms, holding the percentage of lower status of the population constant.

### Question 2.3.1 Bootstrapping the room number coefficient, theta_RM (5 points)

We will follow the same steps as when we bootstrapped the mean in Part 1:

1. Draw random sample of size *k* with replacement from your existing sample.
2. Compute the statistic for the sample (in this case, fit a linear model and extract $\theta_\mathrm{RM}$ from it)
3. Replicate steps 1 and 2 *B* times, to get *B* statistics, forming samples from your bootstrap sampling distribution, $\tilde\theta_\mathrm{RM}$

In [25]:
k = len(housing)
B = 5000
# seed for the random number generator (for reproducibility)
random_state = np.random.RandomState(42)
bootstrap_theta_rm = []
for _ in range(B):
    #get a random sample of k rows from housing *with replacement*
    
    df_sample = housing.sample(n=k, replace=True, random_state=random_state)
    
    #split out predictors (assign them to a data frame called X)
    #and response variable (assign it to a series called y)
    # YOUR CODE HERE
    
    X = pd.DataFrame(df_sample, columns = ['RM', 'LSTAT'])
    y = pd.Series(df_sample['price'])
    
    
    initialize_model=linear_model.LinearRegression()
    
    #fit the model using the linear_regression function we implemented before
    # assign it to the variable `model`
    # YOUR CODE HERE
    model = initialize_model.fit(X,y)
    
    #extract theta_rm coefficient from the fit
    # assign it to the variable `theta_rm`
    # YOUR CODE HERE
    
    theta_rm = model.coef_[0]
    
    #add RM coefficient to list of bootstrap samples
    bootstrap_theta_rm.append(theta_rm)
  

In [26]:
# test your bootstrap code for theta_rm
assert np.abs(np.mean(bootstrap_theta_rm) - 5.124) < .001, "Bootstrap theta_rm: bootstrap mean of theta_rm does not match expected value, check your bootstrapping code"
assert np.abs(np.std(bootstrap_theta_rm) - 0.761) < .001, "Bootstrap theta_rm: bootstrap SD of theta_rm does not match expected value, check your bootstrapping code"

### Question 2.3.2 Visualizing the bootstrap sampling distribution of theta_RM (5 points)

Visualize the bootstrap sampling distribution ($\tilde{\theta}_{RM}$ = `bootstrap_theta_rm`) as a *density plot*, marking the mean with a vertical rule, as we did for the sampling distribution at the end of Part 1:

In [27]:
df = pd.DataFrame(data = {"bootstrap_theta_rm" : bootstrap_theta_rm})

pop_density_chart_theta = alt.Chart(df).transform_density(
    density='bootstrap_theta_rm',
    as_=['bootstrap_theta_rm', 'density'],
    
).mark_area(color="lightgray").encode(
    x=alt.X('bootstrap_theta_rm', axis=alt.Axis(title="Distribution of the the room number coefficient RM")),
    y=alt.Y('density:Q', axis=alt.Axis(title="Density"))
)

pop_mean_chart_theta = alt.Chart(df).mark_rule(color="black", strokeDash=[1,1]).encode(
    x="mean(bootstrap_theta_rm)"
)

pop_chart_theta = pop_density_chart_theta + pop_mean_chart_theta

pop_chart_theta

alt.LayerChart(...)

In [28]:
import numpy as np
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h


mean_confidence_interval(bootstrap_theta_rm)

(5.1245071355276535, 5.103384771353738, 5.145629499701569)

In [29]:
boston_df = pd.DataFrame(boston.data)

x_simple = housing['RM']
y_simple = housing['price']
pearson_correlation = np.corrcoef(x_simple, y_simple)

print(my_rho)

NameError: name 'my_rho' is not defined

### Question 2.3.3 Reflect on your visualization and results (5 points)

How would you describe the results of your model above? Write up a description referencing the above visualization and taking into account your uncertainty. Be sure to focus your analysis on on the uncertainty of the relationship between rooms and price. 

The visualization shown above is a simulated price distribution for 5000 houses in the Boston area. On average, the  coefficient for the room number predictor is about 5.12, indicated by the solid black vertical line and the mean value for the distribution calculated using the function two cells above. This implies that, on average, for each additional room added to a house, the increase in the price of the house will amount to about 5.12(in thousands). However, it would also not be unexpected for the true value of coefficient to vary anywhere within the 95% confidence interval range, from the lower bound of 5.10, to the upper bound of 5.14. It is also worth noting that the relationship between the number of rooms in a house and its price is strongly linear, as indicted by the upward sloping regression line running through the scatter plot distribution in section 2.1, and by the strongly positive pearson correlation coefficient of 0.70 calculated one cell above(between number of rooms and house price). 

Please remember to submit both the HTML and .ipynb formats of your completed notebook. When generating your HTML, be sure to run your complete code first before downloading as HTML.
Please remember to work on your explanations and interpretations!